# Import libraries

In [1]:
import numpy as np
import pandas as pd

import datetime
import requests
from prettytable import PrettyTable

import zipfile

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

# Adding data variables

In [2]:
path_data_variables = 'data/variables_data/'

In [3]:
### Stock exchange indices ###

# WIG20 
# from: https://stooq.com/q/d/?s=wig20&c=0
wig20_d = pd.read_csv(path_data_variables+'wig20_d.csv')

In [4]:
wig20_d

,Date,Open,High,Low,Close,Volume
0,1991-04-16,100.00,100.00,100.00,100.00,325
1,1991-04-23,95.70,95.70,95.70,95.70,5905
2,1991-04-30,93.50,93.50,93.50,93.50,7162
3,1991-05-14,92.90,92.90,92.90,92.90,18300
4,1991-05-21,95.50,95.50,95.50,95.50,14750
...,...,...,...,...,...,...
6989,2021-01-18,1985.59,2023.74,1970.06,2023.74,32644709
6990,2021-01-19,2034.26,2047.11,1980.12,1980.12,32703284
6991,2021-01-20,1983.49,1998.43,1960.82,1986.09,26554080
6992,2021-01-21,1997.58,2007.64,1948.97,1953.49,25603661
